# 🛍️ ShopSense — Phase 6: Session-Based Personalization


## Step 0 — Install Dependencies

In [1]:
import sys
!{sys.executable} -m pip install langgraph langchain langchain-groq sentence-transformers faiss-cpu pandas numpy python-dotenv groq -q
print('✅ Dependencies installed.')


[notice] A new release of pip is available: 25.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
✅ Dependencies installed.


## Step 1 — Load All Components from Previous Phases

In [2]:
import os, json, re, faiss, numpy as np, pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from groq import Groq
from sentence_transformers import SentenceTransformer
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END

load_dotenv()
DATA_DIR   = Path('data')
GROQ_MODEL = 'llama-3.1-8b-instant'

embed_model   = SentenceTransformer('all-MiniLM-L6-v2')
product_index = faiss.read_index(str(DATA_DIR / 'faiss_index.bin'))
groq_client   = Groq(api_key=os.getenv('GROQ_API_KEY'))

with open(DATA_DIR / 'faiss_metadata.json') as f:
    product_metadata = json.load(f)

df_products  = pd.read_csv(DATA_DIR / 'products_clean.csv')
median_price = df_products['price'].median()

print(f'✅ All components loaded')
print(f'   Products : {len(product_metadata):,}')
print(f'   Model    : {GROQ_MODEL}')

/Users/rithikabaskaran/MyFiles/ShopSense/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2603.12it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ All components loaded
   Products : 10,000
   Model    : llama-3.1-8b-instant


## Step 2 — Define the Session State

A session is a single conversation with ShopSense.
It persists from the first query until the user resets it.

The session tracks two types of signals:
- **Explicit**: products the user directly liked or dismissed
- **Implicit**: patterns extracted from search history automatically

In [3]:
from dataclasses import dataclass, field
from collections import Counter

@dataclass
class SessionMemory:
    """
    Stores everything the agent has learned about the user this session.

    Explicit signals (user told us directly):
        liked_asins    : ASINs the user gave 👍
        dismissed_asins: ASINs the user gave 👎

    Implicit signals (extracted from behavior):
        search_history : All queries the user has made this session
        seen_asins     : All products shown to user (avoid repeats)
        price_points   : Prices of products user engaged with
        keyword_counts : Frequency of keywords across all queries
        liked_categories: Categories of liked products
        liked_keywords : Keywords from titles of liked products
        avg_price_liked: Average price of liked products
        turn_count     : How many searches have happened
    """
    # Explicit
    liked_asins     : list  = field(default_factory=list)
    dismissed_asins : list  = field(default_factory=list)

    # Implicit
    search_history  : list  = field(default_factory=list)
    seen_asins      : set   = field(default_factory=set)
    price_points    : list  = field(default_factory=list)
    keyword_counts  : Counter = field(default_factory=Counter)
    liked_categories: list  = field(default_factory=list)
    liked_keywords  : list  = field(default_factory=list)
    avg_price_liked : float = field(default=None)
    turn_count      : int   = field(default=0)

    def to_context_string(self) -> str:
        """
        Converts session memory into a natural language string
        that gets injected into the agent's prompts.
        """
        if self.turn_count == 0:
            return 'No session history yet — this is the first search.'

        parts = []

        if self.search_history:
            recent = self.search_history[-3:]  # last 3 queries
            parts.append(f"Recent searches: {', '.join(repr(q) for q in recent)}")

        if self.liked_asins:
            parts.append(f"User liked {len(self.liked_asins)} product(s) this session")

        if self.dismissed_asins:
            parts.append(f"User dismissed {len(self.dismissed_asins)} product(s) this session")

        if self.liked_keywords:
            top_kw = self.liked_keywords[:5]
            parts.append(f"Keywords from liked products: {', '.join(top_kw)}")

        if self.avg_price_liked is not None:
            parts.append(f"Average price of liked products: ${self.avg_price_liked:.2f}")

        if self.keyword_counts:
            top_words = [w for w, _ in self.keyword_counts.most_common(5)
                        if len(w) > 3]  # skip short words
            if top_words:
                parts.append(f"Most searched keywords: {', '.join(top_words)}")

        return ' | '.join(parts) if parts else 'Session started but no strong preferences yet.'

    def reset(self):
        """Clear all session memory."""
        self.__init__()
        print('🔄 Session reset.')


# Create the global session object
# This persists across all agent calls until explicitly reset
session = SessionMemory()

print('✅ SessionMemory defined and initialized.')
print(f'   Tracks: explicit likes/dismissals + implicit keyword/price signals')
print(f'   Scope : full session (persists until session.reset() is called)')

✅ SessionMemory defined and initialized.
   Tracks: explicit likes/dismissals + implicit keyword/price signals
   Scope : full session (persists until session.reset() is called)


## Step 3 — Define Session Update Functions

These functions update the session memory after each interaction.

In [4]:
import re as re_module

STOPWORDS = {'the', 'a', 'an', 'for', 'and', 'or', 'in', 'on', 'at', 'to',
             'with', 'of', 'is', 'it', 'this', 'that', 'my', 'me', 'i',
             'set', 'inch', 'pack', 'piece', 'size', 'color', 'new'}


def extract_keywords(text: str) -> list[str]:
    """Extract meaningful keywords from a text string."""
    words = re_module.findall(r'\b[a-z]{4,}\b', text.lower())
    return [w for w in words if w not in STOPWORDS]


def update_session_after_search(query: str, results: list[dict]):
    """
    Called automatically after every search.
    Updates implicit signals from the query and results shown.
    """
    # Track the query
    session.search_history.append(query)
    session.turn_count += 1

    # Extract keywords from query
    keywords = extract_keywords(query)
    session.keyword_counts.update(keywords)

    # Track which products were shown
    for product in results:
        session.seen_asins.add(product['asin'])
        if product.get('price'):
            session.price_points.append(float(product['price']))


def record_like(asin: str, product: dict):
    """
    Called when user gives a 👍 to a product.
    Updates explicit + implicit signals from the liked product.
    """
    if asin not in session.liked_asins:
        session.liked_asins.append(asin)

    # Extract keywords from liked product title
    title_keywords = extract_keywords(product.get('title', ''))
    session.liked_keywords.extend(title_keywords[:3])

    # Track category
    if product.get('category'):
        session.liked_categories.append(product['category'])

    # Update average price of liked products
    liked_prices = [
        float(product_metadata[i]['price'])
        for i, m in enumerate(product_metadata)
        if m['asin'] in session.liked_asins
        and m.get('price') and str(m.get('price')) != 'nan'
    ]
    if liked_prices:
        session.avg_price_liked = sum(liked_prices) / len(liked_prices)

    print(f'👍 Liked: {product["title"][:60]}')


def record_dismiss(asin: str, product: dict):
    """
    Called when user gives a 👎 to a product.
    """
    if asin not in session.dismissed_asins:
        session.dismissed_asins.append(asin)
    print(f'👎 Dismissed: {product["title"][:60]}')


print('✅ Session update functions defined:')
print('   update_session_after_search() — called after every search')
print('   record_like()    — called when user gives 👍')
print('   record_dismiss() — called when user gives 👎')

✅ Session update functions defined:
   update_session_after_search() — called after every search
   record_like()    — called when user gives 👍
   record_dismiss() — called when user gives 👎


## Step 4 — Define the Personalized Agent State

Extends Phase 5's AgentState with session context fields.

In [5]:
class PersonalizedAgentState(TypedDict):
    """
    Extended agent state that includes session context.
    All Phase 5 fields plus:
        session_context      : Natural language summary of session memory
        personalization_note : How personalization influenced this search
    """
    # From Phase 5
    query               : str
    intent              : Optional[str]
    filters             : Optional[dict]
    search_results      : Optional[list]
    final_results       : Optional[list]
    clarify_question    : Optional[str]
    explanation         : Optional[str]
    needs_clarification : bool
    # New in Phase 6
    session_context     : Optional[str]
    personalization_note: Optional[str]

print('✅ PersonalizedAgentState defined.')

✅ PersonalizedAgentState defined.


## Step 5 — Redefine Nodes with Session Awareness

Three nodes are updated from Phase 5:
- `analyze_node` — now sees session history when refining the query
- `search_node` — now re-ranks results using session preferences
- `explain_node` — now mentions how personalization influenced results

In [6]:
# ── Helper: clean price ────────────────────────────────────────
def clean_price(p):
    try:
        v = float(p)
        return v if not np.isnan(v) else median_price
    except:
        return median_price


# ── Helper: personalization re-scoring ────────────────────────
def personalize_results(results: list[dict]) -> list[dict]:
    """
    Re-scores and re-ranks results using session preferences.

    Scoring adjustments:
    +0.15  — product keywords match liked product keywords
    +0.10  — product price is close to avg price of liked products
    -0.20  — product was already dismissed by user
    -0.10  — product was already shown to user this session
    """
    if session.turn_count == 0:
        return results  # no session data yet, return as-is

    liked_kw_set = set(session.liked_keywords)

    for product in results:
        score = product.get('similarity_score', 0.0)
        title_kws = set(extract_keywords(product.get('title', '')))

        # Boost if title keywords overlap with liked product keywords
        if liked_kw_set and title_kws & liked_kw_set:
            score += 0.15

        # Boost if price is close to avg liked price (within 30%)
        if session.avg_price_liked:
            price = clean_price(product.get('price'))
            price_diff = abs(price - session.avg_price_liked)
            if price_diff < session.avg_price_liked * 0.3:
                score += 0.10

        # Penalize dismissed products
        if product['asin'] in session.dismissed_asins:
            score -= 0.20

        # Slight penalty for already seen products (prefer novelty)
        if product['asin'] in session.seen_asins:
            score -= 0.10

        product['personalized_score'] = round(score, 4)

    # Re-rank by personalized score
    return sorted(results, key=lambda x: x['personalized_score'], reverse=True)


# ── NODE 1: Analyze (session-aware) ───────────────────────────
def analyze_node(state: PersonalizedAgentState) -> PersonalizedAgentState:
    """Analyzes query with session context to refine intent and filters."""
    query           = state['query']
    session_context = session.to_context_string()

    prompt = f"""You are a shopping assistant analyzing a user query.

Session context (what we know about this user so far):
{session_context}

Current query: "{query}"

Using BOTH the session context and current query, respond with JSON:
{{
  "intent": "search" or "clarify",
  "needs_clarification": true or false,
  "search_query": "refined query incorporating session context if helpful",
  "filters": {{
    "max_price": null or number,
    "min_price": null or number,
    "min_rating": null or number,
    "keyword": null or string
  }},
  "reasoning": "one sentence explaining how session context influenced this"
}}

Rules:
- Use session context to make the search_query more specific if the current query is vague
- If user previously searched for budget items, bias max_price accordingly
- Only clarify if query is completely uninterpretable even with session context
- JSON only, no extra text"""

    response = groq_client.chat.completions.create(
        model=GROQ_MODEL,
        messages=[{'role': 'user', 'content': prompt}],
        max_tokens=250,
        temperature=0.1
    )

    raw = response.choices[0].message.content.strip()

    try:
        clean   = re.sub(r'```json|```', '', raw).strip()
        parsed  = json.loads(clean)
        filters = parsed.get('filters', {})
        return {
            **state,
            'intent'             : parsed.get('intent', 'search'),
            'needs_clarification': parsed.get('needs_clarification', False),
            'query'              : parsed.get('search_query', query),
            'session_context'    : session_context,
            'filters': {
                'max_price' : filters.get('max_price'),
                'min_price' : filters.get('min_price'),
                'min_rating': filters.get('min_rating'),
                'keyword'   : filters.get('keyword')
            }
        }
    except:
        return {**state, 'intent': 'search', 'needs_clarification': False,
                'filters': {}, 'session_context': session_context}


# ── NODE 2: Clarify ───────────────────────────────────────────
def clarify_node(state: PersonalizedAgentState) -> PersonalizedAgentState:
    """Generates clarifying question, informed by session context."""
    context  = state.get('session_context', '')
    response = groq_client.chat.completions.create(
        model=GROQ_MODEL,
        messages=[{'role': 'user', 'content':
            f'Session context: {context}\n'
            f'Query: "{state["query"]}" is still too vague.\n'
            f'Ask ONE clarifying question under 20 words.'
        }],
        max_tokens=60, temperature=0.5
    )
    return {**state, 'clarify_question': response.choices[0].message.content.strip()}


# ── NODE 3: Search (with personalization re-ranking) ──────────
def search_node(state: PersonalizedAgentState) -> PersonalizedAgentState:
    """Semantic search + personalization re-scoring."""
    query = state['query']

    # FAISS search
    q_emb = embed_model.encode(
        [query], normalize_embeddings=True, convert_to_numpy=True
    ).astype('float32')
    scores, indices = product_index.search(q_emb, 20)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx == -1:
            continue
        p = product_metadata[idx].copy()
        p['similarity_score'] = round(float(score), 4)
        p['price'] = clean_price(p.get('price'))
        results.append(p)

    # Apply personalization re-scoring
    personalized = personalize_results(results)

    # Track what was shown
    update_session_after_search(query, personalized[:5])

    return {**state, 'search_results': personalized}


# ── NODE 4: Filter ────────────────────────────────────────────
def filter_node(state: PersonalizedAgentState) -> PersonalizedAgentState:
    """Applies price/rating filters. Unchanged from Phase 5."""
    fi       = state.get('filters') or {}
    products = state.get('search_results') or []

    filtered = products
    if fi.get('max_price'):
        filtered = [p for p in filtered if p.get('price', 0) <= fi['max_price']]
    if fi.get('min_price'):
        filtered = [p for p in filtered if p.get('price', 0) >= fi['min_price']]
    if fi.get('min_rating'):
        filtered = [p for p in filtered if p.get('rating', 0) >= fi['min_rating']]
    if fi.get('keyword'):
        kw = fi['keyword'].lower()
        filtered = [p for p in filtered
                    if kw in p.get('title','').lower()
                    or kw in p.get('description','').lower()]
    if len(filtered) < 3:
        filtered = products[:10]

    return {**state, 'final_results': filtered[:5]}


# ── NODE 5: Explain (personalization-aware) ───────────────────
def explain_node(state: PersonalizedAgentState) -> PersonalizedAgentState:
    """Generates explanation that mentions how personalization helped."""
    query   = state['query']
    results = state.get('final_results') or []
    context = state.get('session_context', '')

    if not results:
        return {**state, 'explanation': 'No products found.',
                'personalization_note': ''}

    summary = '\n'.join([
        f"{i+1}. {r['title'][:60]} (${r.get('price',0):.2f} | {r.get('rating')}⭐)"
        for i, r in enumerate(results)
    ])

    has_session = session.turn_count > 1
    session_instruction = (
        f'Session context: {context}\n'
        f'Briefly mention (1 sentence) how the user\'s history influenced these results.'
        if has_session else
        'This is the user\'s first search — no personalization yet.'
    )

    prompt = f"""You are ShopSense, a helpful shopping assistant.

User searched for: "{query}"
{session_instruction}

Recommended products:
{summary}

Write 3-4 friendly sentences explaining why these match.
If session context influenced results, mention it naturally.
End with one practical shopping tip."""

    response = groq_client.chat.completions.create(
        model=GROQ_MODEL,
        messages=[{'role': 'user', 'content': prompt}],
        max_tokens=180,
        temperature=0.7
    )

    # Generate personalization note separately
    p_note = ''
    if has_session and (session.liked_asins or session.keyword_counts):
        top_kw = [w for w, _ in session.keyword_counts.most_common(3) if len(w) > 3]
        p_note = f"Personalized using: {len(session.search_history)} searches"
        if session.liked_asins:
            p_note += f", {len(session.liked_asins)} liked product(s)"
        if top_kw:
            p_note += f", top keywords: {', '.join(top_kw)}"

    return {
        **state,
        'explanation'        : response.choices[0].message.content.strip(),
        'personalization_note': p_note
    }


print('✅ All 5 session-aware nodes defined.')

✅ All 5 session-aware nodes defined.


## Step 6 — Build the Personalized LangGraph

In [7]:
from langgraph.graph import StateGraph, END

graph = StateGraph(PersonalizedAgentState)

graph.add_node('analyze', analyze_node)
graph.add_node('clarify', clarify_node)
graph.add_node('search',  search_node)
graph.add_node('filter',  filter_node)
graph.add_node('explain', explain_node)

graph.set_entry_point('analyze')

graph.add_conditional_edges(
    'analyze',
    lambda s: 'clarify' if s.get('needs_clarification') else 'search',
    {'clarify': 'clarify', 'search': 'search'}
)

graph.add_edge('clarify', END)
graph.add_edge('search',  'filter')
graph.add_edge('filter',  'explain')
graph.add_edge('explain', END)

personalized_agent = graph.compile()

print('✅ Personalized LangGraph agent compiled.')

✅ Personalized LangGraph agent compiled.


## Step 7 — Build the Interactive Runner
Supports 👍/👎 feedback after each search.

In [8]:
def run_personalized_agent(query: str) -> dict:
    """Run the personalized ShopSense agent."""
    initial_state: PersonalizedAgentState = {
        'query'               : query,
        'intent'              : None,
        'filters'             : {},
        'search_results'      : None,
        'final_results'       : None,
        'clarify_question'    : None,
        'explanation'         : None,
        'needs_clarification' : False,
        'session_context'     : None,
        'personalization_note': None
    }

    print(f'\n{"="*65}')
    print(f'🛍️  ShopSense  |  Turn {session.turn_count + 1}')
    print(f'📝 Query: "{query}"')
    if session.turn_count > 0:
        print(f'🧠 Session: {session.to_context_string()[:80]}...')
    print(f'{"="*65}')

    final_state = personalized_agent.invoke(initial_state)

    # Show agent reasoning
    filters = final_state.get('filters') or {}
    active  = {k: v for k, v in filters.items() if v is not None}
    print(f'\n🧠 Intent : {final_state.get("intent", "search")}')
    print(f'   Filters: {active if active else "none"}')

    if final_state.get('needs_clarification'):
        print(f'\n❓ {final_state["clarify_question"]}')
        return final_state

    # Show results
    results = final_state.get('final_results') or []
    print(f'\n📦 Top {len(results)} Recommendations:')
    print('─' * 65)

    for i, r in enumerate(results, 1):
        p_score = r.get('personalized_score', r.get('similarity_score', 0))
        print(f"{i}. {r['title'][:68]}")
        print(f"   💰 ${r.get('price', 0):.2f}  |  "
              f"⭐ {r.get('rating', 'N/A')} ({int(r.get('rating_count', 0)):,} reviews)")
        print(f"   Score: {p_score}  |  ASIN: {r['asin']}")
        print()

    if final_state.get('explanation'):
        print(f'💬 {final_state["explanation"]}')

    if final_state.get('personalization_note'):
        print(f'\n✨ {final_state["personalization_note"]}')

    return final_state


def give_feedback(result_state: dict, product_index_1based: int, feedback: str):
    """
    Give explicit feedback on a result.

    Args:
        result_state       : The state returned by run_personalized_agent()
        product_index_1based: Which result (1=first, 2=second etc.)
        feedback           : 'like' or 'dismiss'
    """
    results = result_state.get('final_results') or []
    if not results or product_index_1based > len(results):
        print('Invalid product index.')
        return

    product = results[product_index_1based - 1]
    asin    = product['asin']

    if feedback.lower() in ('like', 'liked', '👍', '+'):
        record_like(asin, product)
    elif feedback.lower() in ('dismiss', 'dismissed', '👎', '-'):
        record_dismiss(asin, product)
    else:
        print('Use "like" or "dismiss"')


def show_session_summary():
    """Print a summary of current session memory."""
    print(f'\n{"─"*50}')
    print(f'📊 Session Summary (Turn {session.turn_count})')
    print(f'─' * 50)
    print(f'Searches         : {len(session.search_history)}')
    print(f'Products liked   : {len(session.liked_asins)}')
    print(f'Products dismissed: {len(session.dismissed_asins)}')
    print(f'Products seen    : {len(session.seen_asins)}')
    if session.keyword_counts:
        top = [w for w, _ in session.keyword_counts.most_common(5) if len(w) > 3]
        print(f'Top keywords     : {top}')
    if session.avg_price_liked:
        print(f'Avg liked price  : ${session.avg_price_liked:.2f}')
    print(f'─' * 50)


print('✅ Interactive runner ready.')
print('   run_personalized_agent(query) — run a search')
print('   give_feedback(state, n, "like"/"dismiss") — give feedback')
print('   show_session_summary() — inspect session memory')
print('   session.reset() — start fresh')

✅ Interactive runner ready.
   run_personalized_agent(query) — run a search
   give_feedback(state, n, "like"/"dismiss") — give feedback
   show_session_summary() — inspect session memory
   session.reset() — start fresh


## Step 8 — Test: Full Personalization Session

We simulate a real user session across 4 turns to see how
the agent's recommendations shift based on feedback.

In [9]:
# Make sure session is clean
session.reset()

# ── Turn 1: First search (no personalization yet) ──────────────
state1 = run_personalized_agent('non stick frying pan')

🔄 Session reset.

🛍️  ShopSense  |  Turn 1
📝 Query: "non stick frying pan"

🧠 Intent : search
   Filters: none

📦 Top 5 Recommendations:
─────────────────────────────────────────────────────────────────
1. Anyfish 8 Inch Frying Pan without Lid Nonstick Induction Skillet Sma
   💰 $22.99  |  ⭐ 4.2 (29 reviews)
   Score: 0.6875  |  ASIN: B0B12JWHF2

2. MICHELANGELO Frying Pan with Lid, Nonstick 8 Inch Frying Pan with Ce
   💰 $23.99  |  ⭐ 4.4 (10,637 reviews)
   Score: 0.6424  |  ASIN: B0B6VTWMQW

3. NGORAY Pancake Egg Frying Pan Nonstick Mini - 4 Cups Cast Iron Omele
   💰 $22.99  |  ⭐ 4.1 (19 reviews)
   Score: 0.6303  |  ASIN: B09QL4GJNW

4. EOXHOME 8 Inch Non Stick Stone Coating Cookware Nonstick Skillet Fry
   💰 $22.99  |  ⭐ 4.5 (424 reviews)
   Score: 0.6065  |  ASIN: B08TWKZH3M

5. Copper Frying Pan 11-Inch Non Stick Ceramic Infused Titanium Steel O
   💰 $16.99  |  ⭐ 4.3 (57 reviews)
   Score: 0.6061  |  ASIN: B06XGSWY8V

💬 You're looking for the perfect non-stick frying pan to eleva

In [10]:
# Give feedback on Turn 1 results
# Like result #1, dismiss result #2
give_feedback(state1, 1, 'like')
give_feedback(state1, 2, 'dismiss')
show_session_summary()

👍 Liked: Anyfish 8 Inch Frying Pan without Lid Nonstick Induction Ski
👎 Dismissed: MICHELANGELO Frying Pan with Lid, Nonstick 8 Inch Frying Pan

──────────────────────────────────────────────────
📊 Session Summary (Turn 1)
──────────────────────────────────────────────────
Searches         : 1
Products liked   : 1
Products dismissed: 1
Products seen    : 5
Top keywords     : ['stick', 'frying']
──────────────────────────────────────────────────


In [11]:
# ── Turn 2: Related search (personalization kicks in) ──────────
# Agent should now factor in the liked product from Turn 1
state2 = run_personalized_agent('cookware set for everyday use')


🛍️  ShopSense  |  Turn 2
📝 Query: "cookware set for everyday use"
🧠 Session: Recent searches: 'non stick frying pan' | User liked 1 product(s) this session |...

🧠 Intent : search
   Filters: {'max_price': 50, 'keyword': 'non stick'}

📦 Top 5 Recommendations:
─────────────────────────────────────────────────────────────────
1. Non-stick Coating, Cookware Pots and Pans Set, Frying Pan, Skillet, 
   💰 $22.99  |  ⭐ 4.3 (97 reviews)
   Score: 0.7498  |  ASIN: B08DCDPPCK

2. Nonstick Cookware Set, 20 Piece Tiffany Blue Pots and Pans Set, Cera
   💰 $89.99  |  ⭐ 4.4 (50 reviews)
   Score: 0.7356  |  ASIN: B097XKM965

3. Keidason Silicone Cooking Utensils Set, 12-piece Kitchen Utensil Set
   💰 $27.95  |  ⭐ 4.6 (164 reviews)
   Score: 0.6005  |  ASIN: B09MCVQPBQ

4. Anyfish 8 Inch Frying Pan without Lid Nonstick Induction Skillet Sma
   💰 $22.99  |  ⭐ 4.2 (29 reviews)
   Score: 0.586  |  ASIN: B0B12JWHF2

5. Earth Pan Nonstick Cookware 12-Inch Skillet, Terra Cotta
   💰 $22.99  |  ⭐ 3.3 (49 rev

In [12]:
# Like result #1 from Turn 2
give_feedback(state2, 1, 'like')
show_session_summary()

👍 Liked: Non-stick Coating, Cookware Pots and Pans Set, Frying Pan, S

──────────────────────────────────────────────────
📊 Session Summary (Turn 2)
──────────────────────────────────────────────────
Searches         : 2
Products liked   : 2
Products dismissed: 1
Products seen    : 9
Top keywords     : ['stick', 'frying', 'cookware', 'everyday']
──────────────────────────────────────────────────


In [13]:
# ── Turn 3: Different category — does personalization transfer? ─
state3 = run_personalized_agent('storage bins for kitchen')


🛍️  ShopSense  |  Turn 3
📝 Query: "storage bins for kitchen"
🧠 Session: Recent searches: 'non stick frying pan', 'non stick cookware set for everyday us...

🧠 Intent : search
   Filters: {'max_price': 50, 'keyword': 'kitchen'}

📦 Top 5 Recommendations:
─────────────────────────────────────────────────────────────────
1. Plastic Storage Bins with Lids and Wheels - For Toys Shoes Clothes B
   💰 $22.99  |  ⭐ 4.6 (626 reviews)
   Score: 0.7176  |  ASIN: B082T45XB9

2. Bread Bin for Kitchen Counter with lid,Dry Food Storage Container,ir
   💰 $21.99  |  ⭐ 4.7 (51 reviews)
   Score: 0.6262  |  ASIN: B092DD7BZ2

3. iDesign Renewable Paulownia Wood Collection Storage Bin with Handles
   💰 $15.29  |  ⭐ 4.4 (156 reviews)
   Score: 0.6258  |  ASIN: B097VCC4RP

4. mDesign Plastic Stackable Kitchen Pantry Cabinet, Refrigerator or Fr
   💰 $35.99  |  ⭐ 4.6 (603 reviews)
   Score: 0.6209  |  ASIN: B097WLKHM5

5. BINO Stackable Plastic Organizer Storage Bins, X-Large - 2 Pack - Pa
   💰 $22.99  |  ⭐ 4.7

In [14]:
# ── Turn 4: Vague follow-up — agent uses session to clarify ────
# A vague query that the agent can now interpret using session context
state4 = run_personalized_agent('something similar but cheaper')


🛍️  ShopSense  |  Turn 4
📝 Query: "something similar but cheaper"
🧠 Session: Recent searches: 'non stick frying pan', 'non stick cookware set for everyday us...

🧠 Intent : search
   Filters: {'max_price': 50, 'keyword': 'non stick frying pan'}

📦 Top 5 Recommendations:
─────────────────────────────────────────────────────────────────
1. Sakuchi Frying Pan Set Nonstick Skillet Set Induction Compatible 3 P
   💰 $49.99  |  ⭐ 4.6 (26 reviews)
   Score: 0.721  |  ASIN: B09CTX4B1Z

2. RAVELLI Italia Linea 20 Non Stick Frying Pan, 12.5inch
   💰 $34.99  |  ⭐ 4.2 (50 reviews)
   Score: 0.7118  |  ASIN: B07NLDYDTN

3. Browne (5813832) 12" Standard Weight Non-Stick Fry Pan
   💰 $127.59  |  ⭐ 4.5 (21 reviews)
   Score: 0.7099  |  ASIN: B001EJN3OQ

4. Ayesha Curry Home Collection Nonstick Frying Pan / Fry Pan / Skillet
   💰 $29.99  |  ⭐ 4.5 (1,251 reviews)
   Score: 0.6893  |  ASIN: B07ZFLCPN7

5. Caynel Nonstick Electric Skillet, 13 Inch Round Frying Pan, Aluminum
   💰 $22.99  |  ⭐ 3.7 (11 revie

## Step 9 — Show Full Session Summary

In [15]:
show_session_summary()

print(f'\nFull session context string:')
print(f'  "{session.to_context_string()}"')


──────────────────────────────────────────────────
📊 Session Summary (Turn 4)
──────────────────────────────────────────────────
Searches         : 4
Products liked   : 2
Products dismissed: 1
Products seen    : 19
Top keywords     : ['stick', 'frying', 'everyday', 'cookware', 'storage']
──────────────────────────────────────────────────

Full session context string:
  "Recent searches: 'non stick cookware set for everyday use', 'storage bins for kitchen organization and everyday use', 'non stick frying pan under $50' | User liked 2 product(s) this session | User dismissed 1 product(s) this session | Keywords from liked products: anyfish, frying, without, stick, coating | Most searched keywords: stick, frying, everyday, cookware, storage"


## Step 10 — Save Session Module

In [16]:
session_module = '''
"""
ShopSense Session Module
Manages session-based personalization.
Used by Phase 9 FastAPI backend.

Usage:
    from session import SessionMemory, update_session_after_search
    from session import record_like, record_dismiss
"""
from dataclasses import dataclass, field
from collections import Counter
import re

STOPWORDS = {\'the\', \'a\', \'an\', \'for\', \'and\', \'or\', \'in\', \'on\',
             \'at\', \'to\', \'with\', \'of\', \'is\', \'it\', \'this\', \'that\',
             \'my\', \'me\', \'i\', \'set\', \'inch\', \'pack\', \'piece\'}


def extract_keywords(text):
    words = re.findall(r\'\\b[a-z]{4,}\\b\', text.lower())
    return [w for w in words if w not in STOPWORDS]


@dataclass
class SessionMemory:
    liked_asins     : list    = field(default_factory=list)
    dismissed_asins : list    = field(default_factory=list)
    search_history  : list    = field(default_factory=list)
    seen_asins      : set     = field(default_factory=set)
    price_points    : list    = field(default_factory=list)
    keyword_counts  : Counter = field(default_factory=Counter)
    liked_categories: list    = field(default_factory=list)
    liked_keywords  : list    = field(default_factory=list)
    avg_price_liked : float   = field(default=None)
    turn_count      : int     = field(default=0)

    def to_context_string(self):
        if self.turn_count == 0:
            return \'No session history yet.\'
        parts = []
        if self.search_history:
            parts.append(f\'Recent searches: {\', \'.join(repr(q) for q in self.search_history[-3:])}\' )
        if self.liked_asins:
            parts.append(f\'Liked {len(self.liked_asins)} product(s)\')
        if self.liked_keywords:
            parts.append(f\'Liked keywords: {\', \'.join(self.liked_keywords[:5])}\')
        if self.avg_price_liked:
            parts.append(f\'Avg liked price: ${self.avg_price_liked:.2f}\')
        return \' | \'.join(parts) if parts else \'Session started.\'

    def reset(self):
        self.__init__()
'''

with open('session.py', 'w') as f:
    f.write(session_module)

print('✅ session.py saved — ready for Phase 9 import')

✅ session.py saved — ready for Phase 9 import
